In [ ]:
-- Retail Sales Insights |SQL(With RFM Analys)
-- SECTION 1: INITIAL DATA VALIDATION

#1.Checking that all records have been imported successfully.
SELECT  COUNT(*)
FROM retail_store_sales.retail_store_sales;

-- SECTION 2: DATA STAGING SETUP

#Rename the original table to create a working staging environment
ALTER TABLE retail_store_sales
RENAME TO sales_staging_1;

#Create a second staging table for data cleaning
CREATE TABLE `sales_staging_2` (
  `Transaction_ID` text,
  `Customer_ID` text,
  `Category` text,
  `Item_ID` text,
  `Price_Per_Unit` text,
  `Quantity` text,
  `Total_Spent` text,
  `Payment_Method` text,
  `Location` text,
  `Transaction_Date` datetime DEFAULT NULL,
  `Discount_Applied` text
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

#Insert data from Stage 1 into Stage 2 for cleaning
INSERT INTO sales_staging_2
SELECT	*
FROM sales_staging_1;

-- SECTION 3: DATA CLEANING AND MISSING VALUE HANDLING

#Inspect how missing values appear (they may be represented as '\N' in Excel)
SELECT Item_ID,
       CHAR_LENGTH(Item_ID) AS len,
       HEX(Item_ID) AS hex
FROM sales_staging_2
WHERE Item_ID IS NOT NULL
LIMIT 50;

#Replace '\N' with actual NULLs
UPDATE sales_staging_2
SET Item_ID = NULL,
	Price_Per_Unit=NULL,
    QuantiTy=NULL,
    Total_Spent=NULL,
    Discount_Applied=NULL
WHERE Item_ID = '\\N'
	OR Price_Per_Unit='\\N'
    OR Quantity='\\N'
	OR Total_Spent='\\N'
	OR Discount_Applied='\\N';

#Check for any records missing Customer_ID
SELECT *
FROM sales_staging_2
WHERE Customer_ID IS NULL;

-- SECTION 4: DUPLICATE HANDLING

#Removing  Duplicates
#This will be straightforward because 'Transaction_ID' acts as a primary key.
SELECT Transaction_ID ,COUNT(*) AS trans_ıd_count
FROM sales_staging_2
GROUP BY Transaction_ID
HAVING trans_ıd_count>1;

#View duplicate rows for validation before deleting.#
SELECT *
FROM (
SELECT *,
ROW_NUMBER() OVER (PARTITION BY Transaction_ID
ORDER BY Transaction_ID) AS row_num,
COUNT(*) OVER (PARTITION BY Transaction_ID) AS trans_id_count
FROM sales_staging_2
) duplicates
WHERE trans_id_count > 1;

#Create Stage 3 table for cleaned and deduplicated data
CREATE TABLE `sales_staging_3` (
  `Transaction_ID` text,
  `Customer_ID` text,
  `Category` text,
  `Item_ID` text,
  `Price_Per_Unit` text,
  `Quantity` text,
  `Total_Spent` text,
  `Payment_Method` text,
  `Location` text,
  `Transaction_Date` datetime DEFAULT NULL,
  `Discount_Applied` text,
 `row_num` int,
 `trans_id_count` int
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci

#Insert deduplicated data
INSERT INTO sales_staging_3
SELECT *,
ROW_NUMBER() OVER (PARTITION BY Transaction_ID
ORDER BY Transaction_ID) AS row_num,
COUNT(*) OVER (PARTITION BY Transaction_ID) AS trans_id_count
FROM sales_staging_2;

#Remove duplicate rows
DELETE FROM sales_staging_3
WHERE row_num>1;

#We can drop the helper columns since they’re no longer needed. Removing them will simplify the dataset and help improve the performance of future queries.
ALTER TABLE sales_staging_3
DROP COLUMN row_num;
ALTER TABLE sales_staging_3
DROP COLUMN trans_id_count;
#Checking the final view of the table
SELECT *
FROM sales_staging_3;

#NOTE:
# In this project, I removed all rows containing NULL values in critical columns — specifically, any rows where Item_ID was NULL.
#These rows couldn’t be populated with valid data and would have violated data integrity constraints.
#However, depending on the situation, deletion isn’t always the best option. If there’s a reliable way to populate missing data, that should be considered first.
#Many analysts tend to skip over this decision point too quickly. Generally, there are two main approaches:
#a) Delete rows with null values if the data is critical and cannot be recovered.
#b) Populate (impute) missing values if they can be calculated or derived logically.
#I will also share a separate post on my read.me section to LinkedIn for those interested in diving deeper into this topic.
# Thoughtful handling of missing data can significantly improve the quality,reliability and effectivity of any analysis.

-- SECTION 5: FINAL DATA STANDARDIZATION

#Remove rows with NULL critical values
DELETE FROM sales_staging_3
WHERE Item_ID IS NULL;

#NOTE:
#While reviewing the Excel file, I noticed that the “Category” column was not standardized — for example, “Food” was written as “foo” in some cells.
#To ensure data consistency and avoid incorrect grouping during exploratory analysis, I standardized these values in Category column by correcting “foo” to “Food.”

#Standardize inconsistent category value
UPDATE sales_staging_3
SET Category='Food'
WHERE Category='Foo';

SECTION 6: EXPLORATORY DATA ANALYSIS (EDA)


#PART 1
#Our management team wants to identify which customers are the highest spenders in our stores and determine the product categories on which they spend the most.
#Identify top spenders by total spent
SELECT Customer_ID,Category,Item_ID,Sum(Total_Spent) AS TOTAL_SPENT
FROM sales_staging_3
GROUP BY Customer_ID,Category,Item_ID
ORDER BY Total_Spent DESC;

#NOTE:
#Many analysts,management teams,businesses and projects about sales tend to focus on the biggest customers by looking at the highest revenue they’ve generated.
#However, a single large order after a long period of inactivity can distort the analysis and may give false results.
#Moreover,these types of customers are essentially wild cards—they can make the numbers look impressive but are not always reliable for sustainable growth.
#In reality, businesses should pay attention not only to their top spenders but also to the customers who place frequent small or medium-sized orders.
#These engaged customers often provide steady revenue, build stronger relationships, and offer more predictable demand over time.
#What we have found with SQL is true.However we would like to focus on our frequent customers

#Identify frequent and high-value customers
WITH Average_Order AS (
SELECT Customer_ID,Category,Item_ID,Sum(Total_Spent) AS TOTAL_SPENT,COUNT(Transaction_ID) AS TOTAL_ORDERS,ROUND(SUM(Total_Spent) / COUNT(Transaction_ID), 2) AS AVERAGE_ORDER_VALUE
FROM sales_staging_3
GROUP BY Customer_ID,Category,Item_ID
)
SELECT *
FROM Average_Order
WHERE TOTAL_ORDERS>5
ORDER BY AVERAGE_ORDER_VALUE DESC
LIMIT 10;

#2.After identifying our frequent customers, they want us to determine which products and categories we should prioritize our marketing campaign.

SELECT Item_ID,Category,Payment_Method,Location,SUM(TOTAL_SPENT) as total_rev
FROM sales_staging_3
GROUP BY  Item_ID,Category,Payment_Method,Location
ORDER BY total_rev DESC
LIMIT 10;

Identify top-performing products and categories
WITH Product_Rev AS
(
SELECT Item_ID,Category,Payment_Method,Location,SUM(TOTAL_SPENT) as total_rev
FROM sales_staging_3
GROUP BY  Item_ID,Category,Payment_Method,Location
ORDER BY total_rev DESC
LIMIT 50
)
SELECT Category,COUNT(Category) as frequent_categories,Item_ID,COUNT(Item_ID) AS frequent_ıtems
FROM Product_Rev
GROUP BY Category,Item_ID
ORDER BY frequent_categories DESC
LIMIT 10;

#3.They also would like to know the best possible time to lauch the campaign on by looking past sale trends.(For better visualization, we focused on sales from the year 2024.)

#NOTE:
#As many people may guess,many retailers offer deep discounts after the holiday season to decrease their stock.
#Therefore,the campaign launch date should probably be after new year in retail businesses.
#However, as analysts, we must support our assumptions with factual data.

#Analyze monthly sales trends (e.g., for year 2024)
SELECT
    DATE_FORMAT(Transaction_Date, '%Y-%m') AS month,
    SUM(Total_Spent) AS monthly_revenue,
    COUNT(*) AS monthly_transactions
FROM sales_staging_3
WHERE YEAR(Transaction_Date) = 2024
GROUP BY DATE_FORMAT(Transaction_Date, '%Y-%m')
ORDER BY month;

#4.Finally,they would like to know the best possible way to lauch the campaign.Let's take how many transactıon took place in 2024 for decide that.

SELECT Location, Count(Transaction_ID)
FROM sales_staging_3
WHERE Transaction_Date LIKE '%2024%'
GROUP BY Location

-- SECTION 7: ADVANCED ANALYSIS - RFM SEGMENTATION


#To analyze customer engagement, they also require a RFM (Recency, Frequency, Monetary) analysis on this dataset.
#After calculating the RFM scores, they want us to classify customers into meaningful segments based on their overall RFM performance.
#(Refer to the README file for detailed labeling criteria.)

WITH RFM AS
(
SELECT Customer_ID,count(Transaction_ID)AS Frequency,max(Transaction_Date) as last_purchase,SUM(Total_Spent) AS Monetary,DATEDIFF('2025-01-20',max(Transaction_date)) AS Recency
FROM sales_staging_3
GROUP BY Customer_ID
)
SELECT *,
CASE
 WHEN Recency<=2 THEN 5
 WHEN Recency<=4 THEN 4
 WHEN Recency<=6 THEN 3
 WHEN Recency<=8 THEN 2
ELSE 1
END AS recency_score,
CASE
 WHEN Monetary>=46000 THEN 5
 WHEN Monetary>=44000 THEN 4
 WHEN Monetary>=42000 THEN 3
 WHEN Monetary>=40000 THEN 2
 ELSE 1
END AS monetary_score,
CASE
 WHEN Frequency>=348 THEN 5
 WHEN Frequency>=336 THEN 4
 WHEN Frequency>=324 THEN 3
 WHEN Frequency>=312 THEN 2
ELSE 1
END AS frequency_score,
  (
        CASE
            WHEN Recency <= 2 THEN 5
            WHEN Recency <= 4 THEN 4
            WHEN Recency <= 6 THEN 3
            WHEN Recency <= 8 THEN 2
            ELSE 1
        END +
        CASE
            WHEN Monetary >= 46000 THEN 5
            WHEN Monetary >= 44000 THEN 4
            WHEN Monetary >= 42000 THEN 3
            WHEN Monetary >= 40000 THEN 2
            ELSE 1
        END +
        CASE
            WHEN Frequency >= 348 THEN 5
            WHEN Frequency >= 336 THEN 4
            WHEN Frequency >= 324 THEN 3
            WHEN Frequency >= 312 THEN 2
            ELSE 1
        END
    ) AS rfm_score
FROM RFM
ORDER BY rfm_score DESC
LIMIT 10;






